In [2]:
library(DiffBind)
library(tidyverse)

if (! exists("code.dir")) {
    code.dir <- getwd()
    out.dir <- gsub("codes_local", "4_DiffBind", code.dir)
    setwd(out.dir)
}

In [118]:
samples.sheet <- '2017_PNAS_Li_simplified.csv'

dbObj <- dba(sampleSheet=samples.sheet)
dbObj <- dba.count(dbObj, bUseSummarizeOverlaps=TRUE)
dbObj <- dba.contrast(dbObj, categories=DBA_FACTOR, minMembers=2)
dbObj <- dba.analyze(dbObj, method=DBA_EDGER)
dba.show(dbObj, bContrasts=T)

# Write normalized rpkm dfs
for (i in c(1:4)) {
    i_name <- colnames(dbObj$class)[i]
    i_df <- dbObj$peaks[[i]]
    i_df$Chr <- paste("chr", i_df$Chr, sep="")
    write_csv(i_df, paste(i_name, "norm.csv", sep="_"))
}

# Write differential outputs
res_edger <- dba.report(dbObj, method=DBA_EDGER, contrast = 1, th=1)
res_edger <- as.data.frame(res_edger)
res_edger$`p.value` <- NULL
res_edger$FDR <- NULL
write_csv(res_edger, "2017_PNAS_Li_CD8-IL2_DE.csv")

STAT5B-noIL2-CD8_1 Spleen STAT5B_noIL2 noIL2 invitro 1 bed

STAT5B-IL2-CD8_1 Spleen STAT5B_IL2 IL2 invitro 1 bed

STAT5B-noIL2-CD8_2 Spleen STAT5B_noIL2 noIL2 invitro 2 bed

STAT5B-IL2-CD8_2 Spleen STAT5B_IL2 IL2 invitro 2 bed

Warning message:
“fragmentSize ignored when bUseSummarizeOverlaps is TRUE in dba.count”


,Group1,Members1,Group2,Members2,DB.edgeR
,<fct>,<fct>,<fct>,<fct>,<fct>
1,STAT5B_noIL2,2,STAT5B_IL2,2,666


In [119]:
il2_df <- read_csv("STAT5B-IL2-CD8_1_norm.csv")
noil2_df <- read_csv("STAT5B-noIL2-CD8_1_norm.csv")

il2_df$noil2_RPKM <- noil2_df$RPKM
il2_df$noil2_Reads <- noil2_df$Reads
il2_df$log2FoldChange <- log2((il2_df$RPKM + 0.1) / (il2_df$noil2_RPKM + 0.1))

il2_df$noil2_RPKM <- noil2_df$RPKM
il2_df$noil2_Reads <- noil2_df$Reads
il2_df$log2FoldChange <- log2((il2_df$RPKM + 0.1) / (il2_df$noil2_RPKM + 0.1))

write_csv(il2_df, "2017_PNAS_Li_CD8-IL2_vs_CD8-noIL2_rpkmFoldChange.csv")


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Chr = col_character(),
  Start = col_double(),
  End = col_double(),
  Score = col_double(),
  RPKM = col_double(),
  Reads = col_double(),
  cRPKM = col_double(),
  cReads = col_double()
)



── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Chr = col_character(),
  Start = col_double(),
  End = col_double(),
  Score = col_double(),
  RPKM = col_double(),
  Reads = col_double(),
  cRPKM = col_double(),
  cReads = col_double()
)




In [124]:
il2_promoted_df <- il2_df %>% filter(log2FoldChange > 2) %>%
                   filter(RPKM > 10) %>% select(one_of(c("Chr", "Start", "End")))
write_delim(il2_promoted_df, "2017_PNAS_Li_CD8-IL2_vs_CD8-noIL2_rpkmFoldChange_IL2promoted.bed", 
            delim="\t", col_names=FALSE)